In [ ]:
!pip install requests

In [ ]:
import requests

In [ ]:
# 웹서버에 요청을 보낸다. 
url = "http://127.0.0.1:8080/corona?ServiceKey=aaa"

response = requests.get(url)


In [ ]:
print(response)

In [ ]:
print(response.text)

In [ ]:
url = "http://127.0.0.1:8080/corona"
_param = {
    "ServiceKey" : 'aaa'
}
res = requests.get(url, params=_param)


In [ ]:
print(res)
print(res.text)

In [ ]:
url = 'http://apis.data.go.kr/1262000/CountryPopulationService2/getCountryPopulationList2'

serviceKey = 'dtbWOdJ/Cz5HE0DGLU+CRPe7pOW0NIQBUcGEqsHZaTRiYCI/5+zugwzQjcvuId7NPdg6rUiW+ft3fm7yqyD4pw=='

params ={
    'ServiceKey' : serviceKey, 
    'pageNo' : '1', 
    'numOfRows' : '10', 
    'cond[country_nm::EQ]' : '가나', 
    'cond[country_iso_alp2::EQ]' : 'GH', 
    'returnType' : 'XML' 
}

response = requests.get(url, params=params)

print(response.content)

In [ ]:
!pip install xmltodict

In [ ]:
import xmltodict

In [ ]:
data = xmltodict.parse(response.content)

In [ ]:
from pprint import pprint

In [ ]:
res_data = data['results']['data']['item']

In [ ]:
df1 = pd.DataFrame(res_data[0]['col'])

In [ ]:
df2 = pd.DataFrame(res_data[1]['col'])

In [ ]:
## 두개의 데이터프레임을 조인 결합
pd.merge(df1, df2, on='@name', how='inner')

### 데이터프레임 결합 
1. 유니언 결합
    - 단순하게 행이나 열을 결합 방식
    - concat({데이터프레임의 이름을 1차원 리스트}, axis={0 | 1}, ignore_index={bool})
        - axis 매개변수 : 0인 경우에는 행을 추가하는 결합방식, 1인 경우에는 열을 추가하는 결합 방식
        - ignore_index : 기본값은 False, False인 경우에는 단순하게 결합, True로 변경시에는 axis=0이라면 인덱스를 초기화, axis=1 컬럼을 초기화
2. 조인 결합 
    - 특정한 조건에 맞춰서 열을 추가하는 결합 방식
    - merge({왼쪽의 데이터프레임명}, {오른쪽의 데이터프레임명}, on = {조건으로사용할 컬럼명}, how = {inner|left|right|outer})

In [ ]:
import pandas as pd
import json

In [ ]:
## 국가 5개의 데이터를 로드해서 csv 파일로 저장 
reg_list = ['프랑스', '이탈리아', '독일', '영국', '오스트리아']

In [ ]:
def api_load(_name):
    url = 'http://apis.data.go.kr/1262000/CountryPopulationService2/getCountryPopulationList2'

    serviceKey = 'dtbWOdJ/Cz5HE0DGLU+CRPe7pOW0NIQBUcGEqsHZaTRiYCI/5+zugwzQjcvuId7NPdg6rUiW+ft3fm7yqyD4pw=='

    params ={
        'ServiceKey' : serviceKey, 
        'pageNo' : '1', 
        'numOfRows' : '10', 
        'cond[country_nm::EQ]' : _name, 
        'returnType' : 'JSON' 
    }

    response = requests.get(url, params=params)

    # api에서 받아온 데이터를 json의 형태로 형태를 변환
    res_data = json.loads(response.text)
    # res_data['data']를 데이터프레임으로 변환
    result = pd.DataFrame(res_data['data'])

    return result


In [ ]:
api_load('프랑스')

In [57]:
for i in reg_list:
    # print(i)
    df = api_load(i)
    # csv파일로 저장 (파일명은 국가의 이름으로)
    df.to_csv(f'{i}.csv')

In [71]:
## map({함수명}, {1차원 배열 데이터})
## 람다 함수 : 함수를 선언함과 동시에 호출
a = list(map(
    api_load, 
    reg_list
))

In [72]:
a

[  country_eng_nm country_iso_alp2 country_nm popltn_cn  popltn_cnt  \
 0         France               FR        프랑스      None    67388001   
 1         France               FR        프랑스      None    67158348   
 
                             popltn_cnt_src  popltn_growth  \
 0  ('19) World Bank (최근 수정일 : 2023.12.18.)           0.34   
 1  ('18) World Bank (최근 수정일 : 2023.12.18.)           0.36   
 
                          popltn_growth_src popltn_month  popltn_year  
 0  ('19) World Bank (최근 수정일 : 2023.12.18.)         None         2019  
 1  ('18) World Bank (최근 수정일 : 2023.12.18.)         None         2018  ,
   country_eng_nm country_iso_alp2 country_nm popltn_cn  popltn_cnt  \
 0         Italia               IT       이탈리아      None    59729081   
 1         Italia               IT       이탈리아      None    60421760   
 
                             popltn_cnt_src  popltn_growth  \
 0  ('19) World Bank (최근 수정일 : 2023.12.18.)          -1.15   
 1  ('18) World Bank (최근 수정일 : 2023.12.18